This example shows how to compare product prices using AgentQL with the [Google Colaboratory](https://colab.research.google.com/) environment.

Install [AgentQL](https://pypi.org/project/agentql/) library

In [ ]:
%pip install agentql

Install the Playwright dependency required by AgentQL.

In [ ]:
!playwright install chromium

You can [store](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) keys in Google Colab's secrets.

In [ ]:
import os

from google.colab import userdata

os.environ["AGENTQL_API_KEY"]=userdata.get('AGENTQL_API_KEY')

Run AgentQL script. Please note that an online environment like Google Colab supports **asynchronous version** of AgentQL.

In [ ]:
import agentql
from playwright.async_api import async_playwright
from IPython.display import HTML
from base64 import b64encode

# Set the URL to the desired website
BESTBUY_URL = "https://www.bestbuy.com/site/nintendo-switch-oled-model-w-joy-con-white/6470923.p?skuId=6470923"
TARGET_URL = "https://www.target.com/p/nintendo-switch-oled-model-with-white-joy-con/-/A-83887639#lnk=sametab"
NINTENDO_URL = "https://www.nintendo.com/us/store/products/nintendo-switch-oled-model-white-set/"

# Define the queries to get the product price
PRODUCT_INFO_QUERY = """
{
    nintendo_switch_price(integer)
}
"""

async with async_playwright() as playwright, await playwright.chromium.launch() as browser:

      # Set up the video recording
      video_dir = os.path.abspath("videos")
      context = await browser.new_context(
        record_video_dir="videos/",
        record_video_size={"width": 1280, "height": 720}
        )
      page = await agentql.wrap_async(await context.new_page())
      await page.goto(BESTBUY_URL, timeout= 60000)

      # Use query_data() method to fetch the price from the BestBuy page
      response = await page.query_data(PRODUCT_INFO_QUERY)

      print("Price at BestBuy: ", response["nintendo_switch_price"])

      await page.goto(NINTENDO_URL, timeout= 60000)

      # Use query_data() method to fetch the price from the Nintendo page
      response = await page.query_data(PRODUCT_INFO_QUERY)

      print("Price at Nintendo: ", response["nintendo_switch_price"])

      await page.goto(TARGET_URL, timeout= 60000)

      # Use query_data() method to fetch the price from the Target page
      response = await page.query_data(PRODUCT_INFO_QUERY)

      print("Price at Target: ", response["nintendo_switch_price"])

      # Display the video
      video_files = [f for f in os.listdir(video_dir) if f.endswith('.webm')]
      if video_files:
          video_path = os.path.join(video_dir, video_files[0])
          with open(video_path, 'rb') as f:
            video_bytes = f.read()
            
          video_b64 = b64encode(video_bytes).decode('utf-8')
          video_html = f"""
          <video width="800" controls>
            <source src="data:video/webm;base64,{video_b64}" type="video/webm">
            Your browser does not support the video tag.
          </video>
          """
          display(HTML(video_html))
      else:
          print("No video file was created") 